In [3]:
#Importamos las librerías necesarias:
from bs4 import BeautifulSoup
import requests
import csv
import pandas as pd
#Agregamos un input donde indicaremos el numero de hoja que ultilizaremos para el web scraping
sheet = input("Introduzca número de hoja"+" ")

#Realizamos listas vacías a las cuales se les agragará los datos:
entry_list = []
name_list = []
protein_list = []
gene_list = []
organism_list = []
status_list = []
nomgo_list = []
#Leemos el excel con las abreviaturas de los genes con pandas:
gen = pd.read_excel("genabrev1.xlsx", sheet_name = sheet)
#Convertimos la lectura en una lista:
genabrev = gen["Gen Abrev"].to_list()


#Realizamos un bucle para obtener todos los urls:

for i in genabrev:                                       
    
    url = "https://www.uniprot.org/uniprot/?query="+i+"+bos+taurus&sort=score" #Se irán pasando cada abreviacion de los 
                                                                               #genes a traves de los URL  

    html = requests.get(url).text

    soup = BeautifulSoup(html, "lxml")
    
    #Empezaremos con el scrapeo                             #con este codigo buscaremos en el html "noResultsMessage" 
    empty = soup.find("div", {"id" : "noResultsMessage"}) #lo cual nos indicara que no se encontraron resultados en la busqueda
    #Si no hay resultado de la búsqueda, se harán los siguientes pasos:  
    if empty is not None:
        entry = " "
        name = " "
        protein = " "          #Esto hará que se impriman espacios en blano en cada columna en caso no existan resultados 
        gene = " "             #del gen que se haya indicado
        organism = " "
        status = " "
        nomgo = " "
        
    #En caso de encontrar resultados en la busqueda se procederá a hacer lo siguiente   
    else: 
        entry = soup.find("td", class_="entryID").text     #Buscaremos la clase "entryID" en el html de la página
        if entry is None:                                  #En caso no exista se imprimirá un espacio en blanco
            
            entry = " "

        url_2 = "https://www.uniprot.org/uniprot/"+entry   #Indicaremos que "entry" se agregue al segundo url a scrapear que
                                                           #que es donde tambien se encuentra la información requerida   
        html_2 = requests.get(url_2).text                  #Esto se puede lograr ya qeu por cada gen solo cambia en Entry                 
                                                            #en el URL
        soup_2 = BeautifulSoup(html_2, "lxml")
        #Entry name:
        name = soup.find_all("td", class_= False)[1].text  #Buscaremos el Entry name en el html de la página
        if name is None:                                    #En caso no tenga se imprimirá un espacio en blanco 
            name = " "
        #Protein:
        protein = soup_2.find("div", {"id" : "content-protein"}).text    #continuaremos haciendo lo mismo para la
        if protein is None:                                              #la siguiente informacion requerida c
            protein = " "
        #Gene
        gene = soup_2.find("div", {"id" : "content-gene"}).text
        if gene is None:
            gene = " "
        #Organism
        organism = soup_2.find("div", {"id" : "content-organism"}).text
        if organism is None:
            gene = " "
        #Status    
        status = soup_2.find("a", class_="icon-uniprot reviewed-icon tooltipped")  

        if status is None:
            status = "Unreviewed"               #Para el caso de status colocaremos "unrewieved" en caso no se encuentre lo pedido 
        else:
            status = "Reviewed"                 #y "reviewed" en caso si se encuentre
        #Go - Biological process
        go_list = soup_2.find("ul", class_="noNumbering biological_process")   #Buscaremos la clase indicada
        
        if go_list is not None:                                #En caso de que exista lo pedido 
            for go in go_list:                                 #para cada elemento de "go_list" se imprimiran 
                nomgo=go.a.text                                #la parte de texto de cada uno de ellos        
        else:
            nomgo = " "                                        # Si es que no existe lo indicado se procederá a imprimir un espacio en blanco

#Agregaremos cada sentencia pedida con lo cual construiremos nuestro data frame que almacene toda la información            
    entry_list.append(entry)
    name_list.append(name)
    protein_list.append(protein)
    gene_list.append(gene)
    organism_list.append(organism)
    status_list.append(status)
    nomgo_list.append(nomgo)

#Pasamos todos los datos a un DataFrame con la librería pandas:
genes = {"Gen Abrev" : genabrev, "Entry" : entry_list, "Entry name" : name_list, "Protein" : protein_list, "Gene" : gene_list,
"Organism" : organism_list, "Status" : status_list, "Go - Biological process" : nomgo_list}
            
genes_df = pd.DataFrame(genes, columns = ["Gen Abrev","Entry","Protein","Gene","Organism","Status","Go - Biological process"])
print(genes_df)
#Pasamos el DataFrame a un excel            
genes_df.to_excel("genes_df4.xlsx", sheet_name = sheet, index = False)


Introduzca número de hoja 7
       Gen Abrev       Entry  \
0           FZD3  A0A3Q1MDX6   
1         FBXO16  A0A4W2IPR7   
2   LOC112447957               
3           PNOC      O62647   
4           ELP3      Q2KJ61   
5          NUGGC      E1BLS9   
6         SCARA5      A5PJQ2   
7            PBK  A0A4W2ENR0   
8          ESCO2      A6QNP8   
9         CCDC25      Q3SWY2   
10        SCARA3      F1MMG6   
11           CLU      P00711   
12       TMEM215      A7MB05   
13     LOC539009      E1BGP6   
14      ADAMDEC1      Q3SZQ0   
15         ADAM7      F1MRA8   
16          NEFM      O77788   
17          NEFL      P02548   
18  LOC112447960               
19  LOC100335268               
20         DOCK5  A0A3Q1MC59   
21     MIR2285AE               
22  LOC112447820               
23         GNRH1      Q0VBW7   
24         KCTD9      F1MFU2   
25  LOC112447821               
26         CDCA2      P61287   
27          EBF2      Q08DL5   
28         ASTN2  A0A3Q1MSJ7   
29        TR